# Data Cleaning 

#### 1. Import pandas library.

In [1]:
import pandas as pd

#### 2. Import the users table.

In [2]:
users = pd.read_csv("/Users/Sara/Documents/Bootcamp/Week2/Day9/lab-data-cleaning/data/users.csv", index_col = 0)
users.head()

,userId,Reputation,Views,UpVotes,DownVotes
0,-1,1,0,5007,1920
1,2,101,25,3,0
2,3,101,22,19,0
3,4,101,11,0,0
4,5,6792,1145,662,5


#### 3. Import the posts table. 

In [3]:
posts = pd.read_csv("/Users/Sara/Documents/Bootcamp/Week2/Day9/lab-data-cleaning/data/posts.csv", index_col = 0)
posts.head()

,PostId,userId,Score,ViewCount,CommentCount
0,1,8.0,23,1278.0,1
1,2,24.0,22,8198.0,1
2,3,18.0,54,3613.0,4
3,4,23.0,13,5224.0,2
4,5,23.0,81,NaN,3


#### 4. Define new dataframes for users and posts with the following selected columns:
**users_sliced columns**: userId, Reputation, Views, UpVotes, DownVotes  
**posts_sliced columns**: postId, Score, userId, ViewCount, CommentCount

In [5]:
new_users = users[["userId", "Reputation", "Views", "UpVotes", "DownVotes"]]
new_posts = posts[["PostId", "Score", "userId", "ViewCount", "CommentCount"]]

#### 5. Merge the two dataframes created in the step above, users_sliced and posts_sliced. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [18]:
new_users_post = new_users.merge(right=new_posts, how="inner", on="userId")

#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [12]:
new_users_post.isnull().sum()

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
PostId              0
Score               0
ViewCount       48396
CommentCount        0
dtype: int64

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before going to the next step.

In [23]:
# I will not remove those data because it's half of the dataset
# I could fill them with the mean of the column
import numpy as np
mean_ViewCount = new_users_post["ViewCount"].mean()
new_users_post["ViewCount"].replace((np.nan, mean_ViewCount), inplace =True)

#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [ ]:
new_users_post.dtypes()